# NHS Safety Thermometer - Harm Tables

http://content.digital.nhs.uk/catalogue/PUB21605

Report http://www.content.digital.nhs.uk/catalogue/PUB21604

I'm not totally clear on the interpretation of these tables - this notebook just seeks to explore (mechanically) how we can use them as lookups for data about particular establishments. Whether that is a meaningful thing to do is another matter.

Scroll down to the end for examples of pulling out data related to:

- particular postcode districts
- particular classes of establishiment (for example, care homes)

In [2]:
import pandas as pd

In [3]:
#Get the data file
!mkdir -p data/
!wget -N -P data/ http://content.digital.nhs.uk/catalogue/PUB21605/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx

--2016-10-24 16:01:39--  http://content.digital.nhs.uk/catalogue/PUB21605/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx
Resolving content.digital.nhs.uk... 194.189.27.28
Connecting to content.digital.nhs.uk|194.189.27.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1167406 (1.1M) [application/vnd.ms-excel.12]
Last-modified header missing -- time-stamps turned off.
--2016-10-24 16:01:40--  http://content.digital.nhs.uk/catalogue/PUB21605/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx
Reusing existing connection to content.digital.nhs.uk:80.
HTTP request sent, awaiting response... 200 OK
Length: 1167406 (1.1M) [application/vnd.ms-excel.12]
Saving to: 'data/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx'

nhs-safe-dat-sep-20 100%[=====================>]   1.11M  2.15MB/s   in 0.5s   

2016-10-24 16:01:40 (2.15 MB/s) - 'data/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx' saved [1167406/1167406]



In [3]:
#Preview the sheetnames
xl=pd.ExcelFile('data/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx')
xl.sheet_names

['Cover',
 'Patients',
 'Surveys',
 'Harm Free#',
 'Ulcers#',
 'Falls#',
 'VTE#',
 'UTI#',
 'Harm Free%',
 'Ulcers%',
 'Falls%',
 'VTE%',
 'UTI%']

In [4]:
#Have a peak one of the sheets
df=pd.read_excel('data/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx', sheetname='Harm Free#')
df.dropna(inplace=True,how='all',axis=1)
df.head()

,Organisation,Org Code,Sep15,Oct15,Nov15,Dec15,Jan16,Feb16,Mar16,Apr16,May16,Jun16,Jul16,Aug16,Sep16
0,2GETHER NHS FOUNDATION TRUST,RTQ,102.0,99.0,108.0,97.0,96.0,90.0,90.0,100.0,81.0,102.0,71.0,78.0,81.0
1,5 BOROUGHS PARTNERSHIP NHS FOUNDATION TRUST,RTV,408.0,381.0,424.0,432.0,416.0,446.0,170.0,279.0,422.0,349.0,216.0,307.0,210.0
2,AARON COURT CARE HOME,VLT23,9.0,6.0,8.0,NaN,6.0,8.0,NaN,12.0,8.0,8.0,7.0,NaN,NaN
3,AARONDALE CARE CENTRE,VLYTK,44.0,45.0,43.0,NaN,41.0,35.0,36.0,40.0,43.0,42.0,40.0,43.0,43.0
4,ABBENDON,VLXTD,21.0,21.0,22.0,21.0,20.0,19.0,20.0,21.0,21.0,NaN,22.0,21.0,22.0


In [5]:
#Let's put the data into a simple db - one table for counts, one for percentages
import sqlite3
con_safety = sqlite3.connect("safetythermometer.sqlite")

In [6]:
df.columns

Index(['Organisation', 'Org Code', 'Sep15', 'Oct15', 'Nov15', 'Dec15', 'Jan16',
       'Feb16', 'Mar16', 'Apr16', 'May16', 'Jun16', 'Jul16', 'Aug16', 'Sep16'],
      dtype='object')

In [20]:
#This loader will populate the db
def quickload(typ,table):
    ''' typ: # or %; table:: counts or percentages'''
    for sheet in [name for name in xl.sheet_names if name.endswith(typ)]:
        df=pd.read_excel('data/nhs-safe-dat-sep-2015-sep-2016-harm-tab.xlsx', sheetname=sheet)
        df.dropna(inplace=True,how='all',axis=1)
        df=pd.melt(df,id_vars=['Organisation','Org Code'],var_name='Month',value_name='Value').dropna()
        df['Indicator']=sheet.strip(typ)
        df.set_index(['Organisation','Org Code','Month']).to_sql(con=con_safety, name=table,if_exists='append')

In [56]:
##Comment out and run the following to load the db after throwing away the old one
#!rm safetythermometer.sqlite
#quickload('#','counts')
#quickload('%','percentages')

In [7]:
pd.read_sql_query('SELECT * FROM counts where "Org Code"="RTQ"',con_safety).head()

,Organisation,Org Code,Month,Value,Indicator
0,2GETHER NHS FOUNDATION TRUST,RTQ,Sep15,102.0,Harm Free
1,2GETHER NHS FOUNDATION TRUST,RTQ,Oct15,99.0,Harm Free
2,2GETHER NHS FOUNDATION TRUST,RTQ,Nov15,108.0,Harm Free
3,2GETHER NHS FOUNDATION TRUST,RTQ,Dec15,97.0,Harm Free
4,2GETHER NHS FOUNDATION TRUST,RTQ,Jan16,96.0,Harm Free


The organisation codes aren't very specific - we can't tell from just the code what sort of establishment it refers to or what area it's in. 

In [8]:
#Get a list of org codes in the report for a particular month]
month='Sep15'
orgs_c=pd.read_sql_query('SELECT DISTINCT "Org Code", Organisation FROM counts WHERE Month="{}"'.format(month),con_safety)
orgs_c.head(2)

,Org Code,Organisation
0,RTQ,2GETHER NHS FOUNDATION TRUST
1,RTV,5 BOROUGHS PARTNERSHIP NHS FOUNDATION TRUST


In [9]:
#How many orgs are there?
len(orgs_c)

783

For a wider lookup, we need to resort to various admin tables. I've pulled these into various tables in a `nhsadmin.sqlite` database (described in the *NHS and GP Administrative Data.ipynb* notebook).

I'm going to start by fishing a bit, looking up each code against the different organisation type tables.

In [10]:
con_admin = sqlite3.connect("nhsadmin.sqlite")

lookup=pd.DataFrame()

#For each code
for c in orgs_c['Org Code'].tolist():
    #and for each administrative table
    for t in pd.read_sql_query('SELECT name FROM sqlite_master WHERE type = "table"',con_admin)['name'].tolist():
        #Look up to see if the code exists in the table
        if len(pd.read_sql_query('SELECT "Organisation Code" FROM {} WHERE "Organisation Code"="{}" LIMIT 1'.format(t,c), con_admin)):
            lookup=pd.concat([lookup,pd.DataFrame([{'Org Code':c,'Table':t}])])
lookup.head()

,Org Code,Table
0,RTQ,etrust
0,RTV,etrust
0,VLT23,ecarehomesite
0,VLYTK,ecarehomesite
0,VLXTD,ecarehomesite


In [11]:
#Look to see whether the same code appears in multiple tables
lookup.groupby(['Org Code']).filter(lambda x: len(x) > 1).groupby(['Org Code']).aggregate(lambda x: '_'.join(x['Table']))['Table'].value_counts()

ecarehomesite_ecarehomesucc    72
etrust_ehospice                69
ehospice_enonnhs               12
Name: Table, dtype: int64

So for sake of convenience, let's check codes against tables in order, removing them from a fetch list as they're matched, and ordering the tables at least in part `ecarehomesite`, `ecarehomesucc`, `ehospice`, `enonnhs` and `etrust`, noting (conveniently) that these are alphabetically ordered

In [12]:
tables=pd.read_sql_query('SELECT name FROM sqlite_master WHERE type = "table"',con_admin)['name'].tolist()
tables.sort()
tables

['eabeydispgp',
 'ecarehomehq',
 'ecarehomesite',
 'ecarehomesucc',
 'eccg',
 'eccgsite',
 'edispensary',
 'egdpprac',
 'egparc',
 'egpcur',
 'ehospice',
 'enonnhs',
 'enurse',
 'epcdp',
 'epcmem',
 'epharmacyhq',
 'ephp',
 'ephpsite',
 'epracarc',
 'epraccur',
 'epracmem',
 'etrust']

In [13]:
lookup=pd.DataFrame()

#For each code
for c in orgs_c['Org Code'].tolist():
    #and for each administrative table
    for t in tables:
        #Look up to see if the code exists in the table
        if len(pd.read_sql_query('SELECT "Organisation Code" FROM {} WHERE "Organisation Code"="{}" LIMIT 1'.format(t,c), con_admin)):
            lookup=pd.concat([lookup,pd.DataFrame([{'Org Code':c,'Table':t}])])
            #if it does exist in this table, don't check any other tables
            break
lookup.head()

,Org Code,Table
0,RTQ,etrust
0,RTV,etrust
0,VLT23,ecarehomesite
0,VLYTK,ecarehomesite
0,VLXTD,ecarehomesite


## Missing Data
Ignoring pursuing this for now, other than just to flag up what's missing so we can try to find out why at a later date...

In [14]:
#How many are we missing?
len(orgs_c)-len(lookup)

14

In [15]:
#Which are we missing?
missing= set(orgs_c['Org Code'].tolist())-set(lookup['Org Code'].tolist())
missing

{'5PT',
 '8DW09',
 '8G827',
 '8HN51',
 'A0432',
 'NT507',
 'NTE03',
 'RNE',
 'TAD',
 'TAE',
 'TAF',
 'TAH',
 'VLYFS',
 'VLYOQ'}

In [16]:
df=pd.read_sql_query('SELECT * FROM counts',con_safety)
df[df['Org Code'].isin(list(missing))].groupby(['Organisation','Org Code']).size()

Organisation                                         Org Code
BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST       RNE         65
BLACKWELL VALE CARE HOME                             A0432       65
BRADFORD DISTRICT CARE TRUST                         TAD         65
CAMDEN AND ISLINGTON NHS FOUNDATION TRUST            TAF         65
CARE.UK - SOUTHAMPTON NHS TREATMENT CENTRE           NT507       65
CHARLTON COURT                                       VLYFS       35
CLAREMONT HOSPITAL                                   NTE03       50
KEECH HOSPICE CARE                                   8G827       55
MANCHESTER MENTAL HEALTH AND SOCIAL CARE TRUST       TAE         65
PARTNERSHIPS IN CARE LTD                             8HN51       65
PEXTON GRANGE NURSING HOME                           VLYOQ       55
SHEFFIELD HEALTH & SOCIAL CARE NHS FOUNDATION TRUST  TAH         65
SUFFOLK PCT                                          5PT         65
WILLEN HOSPICE                                       8

## Quick Report By Area

Can we generate some simple reports by area and type of establishment?

Establishments in an area can be found by looking up the establishment code filtered by area in each table though we may need to do some poking around to get the right columns (I maybe need to wrtie some simple table wrappers or normalise column names...)


In [17]:
#What tables do we need to check?
lookup['Table'].unique()

array(['etrust', 'ecarehomesite', 'ehospice', 'ephpsite', 'enonnhs', 'ephp'], dtype=object)

In [18]:
#Metadata describing the tables in the administrative data db is missing - maybe need to add?

tableName={"epraccur":"Current Medical Practices and Prescribing Cost Centres",
"etrust":"NHS Trusts and Trust Sites",
"eccg":"Clinical Commissioning Groups",
"eccgsite":"CCG Sites",
"epcmem":"Current and historical records of membership of CCGs, Primary Care Trusts, Primary Care Groups by General Medical Practice",
"epracmem":"current and historical records of membership of practices by GPs",
"egdpprac":"Dental Surgeries",
"egpcur":"Current General Medical Practitioners (GPs)",
"egparc":"Archived GPs",
"epracarc":"Archived GP Practices",
"ehospice":"Hospices",
"epharmacyhq":"Pharmacy Headquarters",
"edispensary":"Dispensaries",
"enurse":"Nurse Prescribers",
"epcdp":"Private Controlled Drug Prescribers",
"eabeydispgp":"Abeyance and Dispersal GP",
"ecarehomehq":"Care Home Headquarters",
"ecarehomesite":"Care Home Sites",
"ecarehomesucc":"Care Home Successors",
"ephp":"Independent Sector Healthcare Providers",
"ephpsite":"Independent Sector Healthcare Provider Sites",
"enonnhs":"Non-NHS Organisations"}


#Would possibly also be good to add a table with cols: code, table, column
#that maps different codes onto the admin tables and columns in which they appear.
#Also some metadata to make it easier to work out what cols are in each table and how the tables join?

In [19]:
#preview what columns are available for each table that contains codes also in Safety Thermometer report
for t in lookup['Table'].unique():
    print(t,tableName[t],pd.read_sql_query('SELECT * FROM {} LIMIT 1'.format(t), con_admin).columns.tolist())

etrust NHS Trusts and Trust Sites ['Organisation Code', 'Name', 'National Grouping', 'High Level Health Geography', 'Address Line 1', 'Address Line 2', 'Address Line 3', 'Address Line 4', 'Address Line 5', 'Postcode', 'Open Date', 'Close Date', 'Contact Telephone Number', 'Amended Record Indicator', 'GOR Code']
ecarehomesite Care Home Sites ['Organisation Code', 'Name', 'National Grouping', 'High Level Health Geography', 'Address Line 1', 'Address Line 2', 'Address Line 3', 'Address Line 4', 'Address Line 5', 'Postcode', 'Open Date', 'Close Date', 'Parent Organisation Code', 'Join Parent Date', 'Left Parent Date', 'Contact Telephone Number', 'Amended Record Indicator', 'Current Care Organisation', 'Country']
ehospice Hospices ['Organisation Code', 'Name', 'National Grouping', 'High Level Health Geography', 'Address Line 1', 'Address Line 2', 'Address Line 3', 'Address Line 4', 'Address Line 5', 'Postcode', 'Open Date', 'Close Date', 'Organisation Sub-Type Code', 'Contact Telephone Numb

The *National Groupings* are quite a high level geography, more general than CCG or Local Authority areas. (Care Home codes are paired with LA codes in CQC inspection quality reports.)

So we need another way to find sites in the Isle of Wight Local Authority area, for example.

One way is to use the postcode district.

In [175]:
iwPostcodes=["PO30","PO31","PO32","PO33","PO34","PO35","PO36","PO37","PO38","PO39","PO40","PO41"]
#We can generate a regular expressions "startwsith" list of these
#Note the space at the end of the string - this stops us matching MK1 with MK11, MK12 etc
'^{} '.format(' |^'.join(iwPostcodes))

'^PO30 |^PO31 |^PO32 |^PO33 |^PO34 |^PO35 |^PO36 |^PO37 |^PO38 |^PO39 |^PO40 |^PO41 '

In [43]:
etrust=pd.read_sql_query('SELECT "Organisation Code", Name,Postcode FROM {}'.format('etrust'),con_admin)
etrustiw=etrust[etrust['Postcode'].str.contains('^{} '.format(' |^'.join(iwPostcodes)))]
etrustiw.head()

,Organisation Code,Name,Postcode
1226,R1F,ISLE OF WIGHT NHS TRUST,PO30 5TG
1227,R1F01,ST MARY'S HOSPITAL,PO30 5TG
1228,R1F04,FAIRLEE,PO30 2ER
1229,R1F10,ARTHUR WEBSTER CLINIC,PO37 7HZ
1230,R1F15,NEWPORT CLINIC,PO30 1JW


In [58]:
#We can now construct a query that searches the safety table against codes for IW sites of a particular site type
pd.read_sql_query('SELECT * \
                      FROM counts WHERE Month="{}" AND "Org Code" IN ("{}")\
                      '.format(month,'","'.join(etrustiw["Organisation Code"].tolist())),con_safety)


,Organisation,Org Code,Month,Value,Indicator
0,ISLE OF WIGHT NHS TRUST,R1F,Sep15,556.0,Harm Free
1,ISLE OF WIGHT NHS TRUST,R1F,Sep15,23.0,Ulcers
2,ISLE OF WIGHT NHS TRUST,R1F,Sep15,1.0,Falls
3,ISLE OF WIGHT NHS TRUST,R1F,Sep15,5.0,VTE
4,ISLE OF WIGHT NHS TRUST,R1F,Sep15,1.0,UTI


In [206]:
#More generally
def reporter_postcodes(samplePostcodes):
    results={}
    for t in lookup['Table'].unique():
        tmp=pd.read_sql_query('SELECT "Organisation Code", Name,Postcode FROM {}'.format(t),con_admin)
        if isinstance(samplePostcodes,list):
            pcl='^{} '.format(' |^'.join(samplePostcodes))
        else:
            #Also support a such on eg PO, PR more generally?
            pcl='^{}'.format(samplePostcodes)
        tmpcodes=tmp[tmp['Postcode'].str.contains(pcl)]
        samplecodelist='","'.join(tmpcodes["Organisation Code"].tolist())
        q='SELECT * \
                          FROM counts WHERE Month="{}" AND "Org Code" IN ("{}")\
                          '.format(month,samplecodelist)
        tmpdf=pd.read_sql_query(q,con_safety)
        #results=pd.concat([results,tmpdf])
        results[t]=tmpdf[tmpdf['Value']>0]
    return results


In [194]:
reporter_postcodes(iwPostcodes)

{'ecarehomesite': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'ehospice': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'enonnhs': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'ephp': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'ephpsite': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'etrust':               Organisation Org Code  Month  Value  Indicator
 0  ISLE OF WIGHT NHS TRUST      R1F  Sep15  556.0  Harm Free
 1  ISLE OF WIGHT NHS TRUST      R1F  Sep15   23.0     Ulcers
 2  ISLE OF WIGHT NHS TRUST      R1F  Sep15    1.0      Falls
 3  ISLE OF WIGHT NHS TRUST      R1F  Sep15    5.0        VTE
 4  ISLE OF WIGHT NHS TRUST      R1F  Sep15    1.0        UTI}

In [207]:
reporter_postcodes('PO')

{'ecarehomesite': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'ehospice': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'enonnhs': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'ephp': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [], 'ephpsite':                             Organisation Org Code  Month  Value  Indicator
 0  NUFFIELD HEALTH - CHICHESTER HOSPITAL    NT212  Sep15    5.0  Harm Free
 1            SPIRE - PORTSMOUTH HOSPITAL    NT305  Sep15   15.0  Harm Free, 'etrust':                                Organisation Org Code  Month  Value  Indicator
 0                   ISLE OF WIGHT NHS TRUST      R1F  Sep15  556.0  Harm Free
 1            PORTSMOUTH HOSPITALS NHS TRUST      RHU  Sep15  978.0  Harm Free
 2   SUSSEX PARTNERSHIP NHS FOUNDATION TRUST      RX2  Sep15  205.0  Harm Free
 3                   ISLE OF WIGHT

## Report Against Site Type
Also do searches by type, eg care homes, across all areas

In [199]:
def reporter_sitetype(typ):
    tmp=pd.read_sql_query('SELECT "Organisation Code", Name,Postcode FROM {}'.format(typ),con_admin)
    samplecodelist='","'.join(tmp["Organisation Code"].tolist())
    q='SELECT * \
                      FROM counts WHERE Month="{}" AND "Org Code" IN ("{}")\
                      '.format(month,samplecodelist)
    results=pd.read_sql_query(q,con_safety)
    results=results[results['Value']>0]
    results=pd.merge(results,tmp[['Organisation Code','Postcode']],left_on='Org Code',right_on='Organisation Code')
    return results

In [200]:
ch=reporter_sitetype('ecarehomesite')
ch.head()

,Organisation,Org Code,Month,Value,Indicator,Organisation Code,Postcode
0,AARON COURT CARE HOME,VLT23,Sep15,9.0,Harm Free,VLT23,CH65 8EU
1,AARONDALE CARE CENTRE,VLYTK,Sep15,44.0,Harm Free,VLYTK,PR7 4PF
2,ABBENDON,VLXTD,Sep15,21.0,Harm Free,VLXTD,PR8 6PE
3,ABBEY GRANGE NURSING HOME,VLXR8,Sep15,2.0,Harm Free,VLXR8,S5 6UU
4,ABBEY GRANGE NURSING HOME,VLXR8,Sep15,1.0,Falls,VLXR8,S5 6UU


In [201]:
#group on popular postcode district
ch['pcDistrict']=ch.apply(lambda x: x['Postcode'].split(' ')[0],axis=1 )
ch.groupby(['pcDistrict']).size().sort_values(ascending=False)

pcDistrict
S8      9
WN8     9
BL8     9
PR8     9
S4      9
PR2     8
LS28    8
TA1     8
BB5     7
LA9     7
FY8     7
CA28    6
LA1     6
PE37    6
M27     6
S5      6
L40     5
CW1     5
TA21    5
PR9     5
DE7     5
PR7     5
BS26    5
LS6     5
LS8     5
WV3     5
WA9     5
OL11    5
OL1     5
L25     4
       ..
OL13    1
PR26    1
S12     1
S2      1
S35     1
S36     1
OL8     1
L23     1
SK15    1
IP25    1
IP20    1
HD2     1
GU27    1
OL16    1
OL12    1
TA2     1
FY5     1
M41     1
DN6     1
DN15    1
DL2     1
DH7     1
SN15    1
DE11    1
CW9     1
LS20    1
NR8     1
LA14    1
NR4     1
BA11    1
dtype: int64

In [204]:
#So what's happening in Preston?
reporter_postcodes(['PR8'])

{'ecarehomesite':                Organisation Org Code  Month  Value  Indicator
 0                  ABBENDON    VLXTD  Sep15   21.0  Harm Free
 1             BIRKDALE PARK    VLNGA  Sep15   21.0  Harm Free
 2     KINGSLEY NURSING HOME    VLNGW  Sep15   20.0  Harm Free
 3   TUDOR BANK NURSING HOME    VLNHA  Sep15   36.0  Harm Free
 4          WESTCLIFFE MANOR    VLNHD  Sep15   22.0  Harm Free
 5      WILLOWS NURSING HOME    VLYQY  Sep15   21.0  Harm Free
 8     KINGSLEY NURSING HOME    VLNGW  Sep15    1.0     Ulcers
 10         WESTCLIFFE MANOR    VLNHD  Sep15    2.0     Ulcers
 11     WILLOWS NURSING HOME    VLYQY  Sep15    2.0     Ulcers,
 'ehospice': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [],
 'enonnhs': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [],
 'ephp': Empty DataFrame
 Columns: [Organisation, Org Code, Month, Value, Indicator]
 Index: [],
 'ephpsite': Empty DataFrame
 Columns: [Organisation, 

So we need to distinguish *Harm Free* as a good thing?

For generating signals, maybe look to percentages in bad things?